In [1]:
!pip install adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.5 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 47.4 MB/s 
     |████████████████████████████████| 3.3 MB 40.6 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 264 kB 3.1 MB/s 
     |████████████████████████████████| 119 kB 45.2 MB/s 
     |████████████████████████████████| 243 kB 45.1 MB/s 


In [2]:
cd /content/drive/MyDrive/CAMemBERT2

/content/drive/MyDrive/CAMemBERT2


In [3]:
from transformers import AutoModelWithHeads,AutoConfig,Trainer,TrainingArguments,EarlyStoppingCallback
import numpy as np
import pdb
from sklearn.metrics import f1_score,accuracy_score,fbeta_score,classification_report
from trainer_utils import get_dataset
import torch
import pandas as pd

dataset_obj,dataset_dict = get_dataset(['ner'],max_length=512)
# prepare for model
dataset_dict = dataset_dict.remove_columns(['dataset','scores'])

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

...

  0%|          | 0/266 [00:00<?, ?ex/s]

  0%|          | 0/243 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
lr = 5e-4
bs = 8
config = AutoConfig.from_pretrained('distilroberta-base',num_labels=9)
model = AutoModelWithHeads.from_pretrained('distilroberta-base',config=config)
adaptors_name='ner'
model.add_adapter(adaptors_name)
model.add_tagging_head(
            adaptors_name,
            num_labels=9,
            overwrite_ok=True
        )
model.train_adapter(adaptors_name)

def compute_metrics(p):
    logits, labels = p.predictions.flatten() , p.label_ids.flatten()
    mask = labels!=-100
    logits , labels = logits[mask] , labels[mask]
    print(classification_report(labels, logits, digits=9))
    report_output = classification_report(labels, logits, digits=9, output_dict=True)
    metrics_dic = {
      f'accuracy' : accuracy_score(labels, logits),
      f'f1_score_avg' : report_output['accuracy'],
      f'f1_score_macro' : report_output['macro avg']['f1-score'],
      f'f1_score_weighted' : report_output['weighted avg']['f1-score'],
    }
    return metrics_dic

class NerLevelTrainer(Trainer):

    loss_fct = torch.nn.CrossEntropyLoss()

    def compute_loss(self,model,inputs,return_outputs=False):
        logits = model(**inputs).logits
        labels = inputs['labels']
        # Only keep active parts of the loss
        active_loss = inputs['attention_mask'].view(-1) == 1
        active_logits = logits.view(-1, model.config.num_labels)
        active_labels = torch.where(active_loss, labels.view(-1), torch.tensor(self.loss_fct.ignore_index).type_as(labels))
        non_padded_mask = active_labels!=-100
        loss = self.loss_fct(active_logits[non_padded_mask], active_labels[non_padded_mask])
        active_preds_mask = torch.logical_and(active_loss,non_padded_mask)
        return (loss,(loss,torch.argmax(logits,2))) if return_outputs else loss

label_names = ['labels']
args = TrainingArguments(
        output_dir = '/content',
        save_total_limit = 1,
        evaluation_strategy = "epoch",
        learning_rate = lr,
        per_device_train_batch_size=bs,
        per_device_eval_batch_size=bs,
        remove_unused_columns=False,
        num_train_epochs = 15,
        weight_decay = 0.01,
        save_strategy = 'epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1_score_avg',
        label_names=label_names
    )

trainer = NerLevelTrainer(
    model=model,
    args=args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    compute_metrics=compute_metrics
    )

early_stopping = EarlyStoppingCallback(early_stopping_patience = 3)
trainer.add_callback(early_stopping)
trainer.train()
trainer.evaluate(dataset_dict['test'])
pd.DataFrame(trainer.state.log_history).to_csv(f'results/neradapters_distilroberta_lr{lr}_bs{bs}_norm.csv')

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score Avg,F1 Score Macro,F1 Score Weighted
1,No log,0.095127,0.968634,0.968634,0.797116,0.967552
2,No log,0.055401,0.984580,0.984580,0.906693,0.984454
3,No log,0.049816,0.985495,0.985495,0.910834,0.985459
4,0.169000,0.040041,0.988922,0.988922,0.935734,0.988913
5,0.169000,0.038030,0.989681,0.989681,0.939378,0.989700
6,0.169000,0.038174,0.989759,0.989759,0.940180,0.989785
7,0.169000,0.035835,0.990440,0.990440,0.944044,0.990435
8,0.033400,0.037632,0.989486,0.989486,0.939207,0.989522
9,0.033400,0.035420,0.990771,0.990771,0.945600,0.990745
10,0.033400,0.035735,0.990927,0.990927,0.947418,0.990906


***** Running Evaluation *****
  Num examples = 266
  Batch size = 8


              precision    recall  f1-score   support

           0  0.994489072 0.996000842 0.995244383     42759
           1  0.961725067 0.968512486 0.965106843      1842
           2  0.972685888 0.980872226 0.976761905      1307
           3  0.801824212 0.721103654 0.759324696      1341
           4  0.736077482 0.809587217 0.771084337       751
           5  0.757467677 0.924877518 0.832843137      1837
           6  0.685106383 0.626459144 0.654471545       257
           7  0.775541796 0.543383948 0.639030612       922
           8  0.770334928 0.465317919 0.580180180       346

    accuracy                      0.968634399     51362
   macro avg  0.828361389 0.781790551 0.797116404     51362
weighted avg  0.968484922 0.968634399 0.967552053     51362



...

              precision    recall  f1-score   support

           0  0.998151874 0.997848406 0.998000117     42759
           1  0.985915493 0.988056460 0.986984816      1842
           2  0.990811639 0.990053558 0.990432453      1307
           3  0.954204204 0.947800149 0.950991395      1341
           4  0.958677686 0.926764314 0.942450914       751
           5  0.973870441 0.973870441 0.973870441      1837
           6  0.942965779 0.964980545 0.953846154       257
           7  0.890041494 0.930585683 0.909862142       922
           8  0.850877193 0.841040462 0.845930233       346

    accuracy                      0.991666991     51362
   macro avg  0.949501756 0.951222224 0.950263185     51362
weighted avg  0.991724268 0.991666991 0.991682403     51362



Saving model checkpoint to /content/checkpoint-1862
Configuration saved in /content/checkpoint-1862/ner/adapter_config.json
Module weights saved in /content/checkpoint-1862/ner/pytorch_adapter.bin
Configuration saved in /content/checkpoint-1862/ner/head_config.json
Module weights saved in /content/checkpoint-1862/ner/pytorch_model_head.bin
Configuration saved in /content/checkpoint-1862/ner/head_config.json
Module weights saved in /content/checkpoint-1862/ner/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 266
  Batch size = 8


              precision    recall  f1-score   support

           0  0.998198493 0.997801632 0.998000023     42759
           1  0.985915493 0.988056460 0.986984816      1842
           2  0.990818669 0.990818669 0.990818669      1307
           3  0.955022489 0.950037286 0.952523364      1341
           4  0.956462585 0.936085220 0.946164199       751
           5  0.970763400 0.976047904 0.973398480      1837
           6  0.928838951 0.964980545 0.946564885       257
           7  0.898002103 0.926247289 0.911906033       922
           8  0.864864865 0.832369942 0.848306333       346

    accuracy                      0.991783809     51362
   macro avg  0.949876339 0.951382772 0.950518534     51362
weighted avg  0.991807551 0.991783809 0.991785828     51362



Saving model checkpoint to /content/checkpoint-1995
Configuration saved in /content/checkpoint-1995/ner/adapter_config.json
Module weights saved in /content/checkpoint-1995/ner/pytorch_adapter.bin
Configuration saved in /content/checkpoint-1995/ner/head_config.json
Module weights saved in /content/checkpoint-1995/ner/pytorch_model_head.bin
Configuration saved in /content/checkpoint-1995/ner/head_config.json
Module weights saved in /content/checkpoint-1995/ner/pytorch_model_head.bin
Deleting older checkpoint [/content/checkpoint-1729] due to args.save_total_limit
Deleting older checkpoint [/content/checkpoint-1862] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best adapter(s) from /content/checkpoint-1995 (score: 0.9917838090417039).
Loading module configuration from /content/checkpoint-1995/ner/adapter_config.json
Overwriting existing adapter 'ner'.
Loading module weights from /content/checkpoint-1995/ner/pyto

              precision    recall  f1-score   support

           0  0.997717015 0.992119615 0.994910442     38323
           1  0.975746269 0.970315399 0.973023256      1617
           2  0.979522184 0.993079585 0.986254296      1156
           3  0.907473310 0.921131848 0.914251569      1661
           4  0.886178862 0.913772455 0.899764151       835
           5  0.934131737 0.935251799 0.934691432      1668
           6  0.788590604 0.914396887 0.846846847       257
           7  0.789893617 0.846153846 0.817056396       702
           8  0.596428571 0.773148148 0.673387097       216

    accuracy                      0.981737913     46435
   macro avg  0.872853574 0.917707731 0.893353943     46435
weighted avg  0.982815214 0.981737913 0.982169625     46435



In [5]:
model.save_adapter('adapters/ner/','ner')

Configuration saved in adapters/ner/adapter_config.json
Module weights saved in adapters/ner/pytorch_adapter.bin
Configuration saved in adapters/ner/head_config.json
Module weights saved in adapters/ner/pytorch_model_head.bin


In [6]:
dataset_dict['train']['attention_mask'][:10]

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [7]:
from sklearn.metrics import classification_report